In [151]:
import pandas as pd
import numpy as np
import os

from utils import *

In [2]:
def mse_AIFrenz(y_true, y_pred):
    '''
    y_true: 실제 값
    y_pred: 예측 값
    '''
    diff = abs(y_true - y_pred)
    
    less_then_one = np.where(diff < 1, 0, diff)
    
    # multi-column일 경우에도 계산 할 수 있도록 np.average를 한번 더 씌움
    score = np.average(np.average(less_then_one ** 2, axis = 0))
    
    return score

In [51]:
from sklearn import ensemble

In [132]:
df = pd.read_csv('./data/train.csv')
df_tst = pd.read_csv('./data/test.csv')
subm = pd.read_csv('./data/sample_submission.csv')

In [133]:
df_trn = df[df['Y18'].isna()]

In [134]:
df_val = df[df['Y00'].isna()]

In [135]:
df_trn

,id,X00,X01,X02,X03,X04,X05,X06,X07,X08,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
0,0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,...,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,NaN
1,1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,...,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,NaN
2,2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,...,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,NaN
3,3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,...,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,NaN
4,4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,...,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,4315,19.5,987.8,1.7,0.4,0.0,1007.8,988.0,21.3,1007.8,...,19.5,20.0,19.0,20.5,20.5,20.0,20.0,20.5,20.0,NaN
4316,4316,19.3,987.7,2.1,0.9,0.0,1007.8,988.1,21.3,1008.0,...,19.5,20.0,19.0,20.5,20.5,20.0,19.5,20.5,19.5,NaN
4317,4317,19.5,987.8,0.9,1.3,0.0,1007.8,988.3,21.2,1008.1,...,19.5,20.0,19.0,20.0,20.0,19.5,19.5,20.0,19.0,NaN
4318,4318,20.0,987.8,1.4,0.5,0.0,1007.8,988.3,21.1,1008.1,...,19.5,20.0,18.5,20.0,20.0,19.5,19.5,20.0,19.0,NaN


In [136]:
X1 = ['X00', 'X07', 'X28', 'X31', 'X32']
X2 = ['X01', 'X06', 'X22', 'X27', 'X29']
X3 = ['X02', 'X03', 'X18', 'X24', 'X26']
X4 = ['X04', 'X10', 'X21', 'X36', 'X39']
X5 = ['X05', 'X08', 'X09', 'X23', 'X33']
X6 = ['X11', 'X14', 'X16', 'X19', 'X34']
X7 = ['X12', 'X20', 'X30', 'X37', 'X38']
X8 = ['X13', 'X15', 'X17', 'X25', 'X35']

Xs = np.concatenate([X1, X2, X3, X4, X5, X6, X7, X8])

Ys = ['Y{}'.format(str(i).zfill(2)) for i in range(18)]


In [137]:
df_trn['Y'] = df_trn.loc[:, Ys].mean(axis=1)

C:\Users\dhzns\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [138]:
mdl = ensemble.RandomForestRegressor()

In [139]:
mdl.fit(df_trn[Xs], df_trn['Y'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [140]:
pred = mdl.predict(df_val[Xs])

In [141]:
pred_adj = pred + 0.55

In [142]:
mse_AIFrenz(df_val['Y18'], pred_adj)

7.472998508409068

In [143]:
len(pred_adj)

432

In [145]:
pred_test = mdl.predict(df_tst[Xs])

In [146]:
pred_test_adj = pred_test+0.55

In [178]:
writeSubm(pred_test_adj)

'0307020118'